# Solve a given chemical equation

## Task-1: Design a input scheme for the user to input a chemical equation that need to be balanced.



In [1]:
import re

In [2]:
string="H 2 + O 2 = H 2 O"

In [3]:
atomList = set(re.split('[ 0-9+=]+',string))
print(atomList)

{'H', 'O'}


In [4]:
#[reactants, products] = (string.split(sep="="))
[reactants, products] = (re.split('=',string))
print(reactants)
print(products)

H 2 + O 2 
 H 2 O


In [5]:
react_list= reactants.split(sep="+")
prod_list = products.split(sep="+")
print(str(react_list),"   ",str(prod_list))

['H 2 ', ' O 2 ']     [' H 2 O']


In [6]:
for chem in prod_list:
    c=list(filter(None,(re.split(' ',chem))))
    print(c)

['H', '2', 'O']


# TASK 1a: Write a function that will take one chemical (reactant or product) and return chemical_composition_vector

Consider reaction ' H 2 + O 2 = H 2 O '
- set atomList is {'O','H'}
- For ' H 2 ', vector is [0, 2]
- For ' O 2 ', vector is [2, 0]
- For 'H 2 O', vector is [-1, -2] (minus sign because of H2O being product

In [7]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [8]:
def get_chemicalVector(chemical, atomList):
    print('chemical is', chemical)
    c = list(filter(None,(re.split(' ',chemical))))
    c_size = len(c)
    c_idx = 0
    c_dict = {}
    print('c= ',c,' c_size=',c_size)
    
    while c_idx < c_size:
        c1 = c[c_idx]
        print('atom is: ',c1)
        if (c1 in atomList): # check if atom is in the atomList
            if ( c_idx  == c_size -1):
                num = 1
                c_idx += 1
            elif  ( c[c_idx +1] in atomList ):
                num = 1
                c_idx += 1
            elif ( (c[c_idx +1 ]).isdigit() ):
                num = int( c[c_idx+1])
                c_idx += 2
            c_dict[c1] = num
            print('chemical ',c1, c_dict, 'c_idx=',c_idx)
        else:
            return "ERROR"
    
    c_vec = np.zeros( (len(atomList),) )
    for c1 in c_dict.keys():
        c_vec[ atomList.index(c1) ] = c_dict[c1]
    print("chemical ", chemical," c_vec ", c_vec)
    return c_vec
            

In [9]:
get_chemicalVector('H 2 S O 4',['H','S','O'])

chemical is H 2 S O 4
c=  ['H', '2', 'S', 'O', '4']  c_size= 5
atom is:  H
chemical  H {'H': 2} c_idx= 2
atom is:  S
chemical  S {'H': 2, 'S': 1} c_idx= 3
atom is:  O
chemical  O {'H': 2, 'S': 1, 'O': 4} c_idx= 5
chemical  H 2 S O 4  c_vec  [2. 1. 4.]


array([2., 1., 4.])

## Task-2: From the input, get into a form that is suitable for mathematical manipulations



1. Lets say that you have atom types $A_1, A_2, \cdots A_N$. 
2. Write each chemical component as a $N$ dimensional vector 
3. Suppose the balanced the chemical reaction is where $R_i$ are reactants and $P_i$ are products, and we require that $r_i$ and $p_i$ be integers... 
$$ r_1 R_1 + r_2 R_2 +\cdots - (p_1 P_1 + p_1 P_2 ) = 0$$ 
3. Set up a system of $N$ simulatneous equations, and solve for it to get $r_i$ and $p_i$
4. What are the conditions under which there is NO solution? 

In [10]:
rxn2 = ' H 2 O + S O 2 + O 2 = H 2 S O 4 '

In [11]:
atomList = list(set(filter(None,re.split('[ 0-9+=]+',rxn2))))
print('atomList = ',atomList)

atomList =  ['S', 'H', 'O']


In [12]:
#[reactants, products] = (string.split(sep="="))
[reactants, products] = (re.split('=',rxn2))
print(reactants)
print(products)

 H 2 O + S O 2 + O 2 
 H 2 S O 4 


In [13]:
react_list= reactants.split(sep="+")
prod_list = products.split(sep="+")
print(str(react_list),"   ",str(prod_list))

[' H 2 O ', ' S O 2 ', ' O 2 ']     [' H 2 S O 4 ']


In [14]:
get_chemicalVector(prod_list[0],atomList)
print(atomList)

chemical is  H 2 S O 4 
c=  ['H', '2', 'S', 'O', '4']  c_size= 5
atom is:  H
chemical  H {'H': 2} c_idx= 2
atom is:  S
chemical  S {'H': 2, 'S': 1} c_idx= 3
atom is:  O
chemical  O {'H': 2, 'S': 1, 'O': 4} c_idx= 5
chemical   H 2 S O 4   c_vec  [1. 2. 4.]
['S', 'H', 'O']


## Task-3: Using the solution above, output the balanced equation

In [15]:
import numpy
from sympy import Matrix


def balance_react_and_prod(reacts, prods, atoms):
    coeff_matrix = []
    for molecule in reacts:
        cvec = get_chemicalVector(molecule, atoms)
        if isinstance(cvec, str):
            raise ValueError("Could not balance the equation")

        coeff_matrix.append(cvec)

    for molecule in prods:
        cvec = get_chemicalVector(molecule, atoms)
        if isinstance(cvec, str):
            raise ValueError("Could not balance the equation")

        coeff_matrix.append(-cvec)

    try:
        solution_space = Matrix(numpy.array(coeff_matrix).transpose()).nullspace()[0]
    except Exception:
        raise ValueError("Could not balance the equation")

    react_cnt = len(reacts)
    solution_str = (
        " + ".join(f"{solution_space[i]} {m}" for i, m in enumerate(reacts))
        + " = "
        + " + ".join(
            f"{solution_space[i + react_cnt]} {m}" for i, m in enumerate(prods)
        )
    )
    return solution_space, solution_str


solution_vector, solution_str = balance_react_and_prod(react_list, prod_list, atomList)
print("The solution (as a vector) for the above equation is:")
print(repr(solution_vector))
print("The balanced chemical equation is:")
print(solution_str)


chemical is  H 2 O 
c=  ['H', '2', 'O']  c_size= 3
atom is:  H
chemical  H {'H': 2} c_idx= 2
atom is:  O
chemical  O {'H': 2, 'O': 1} c_idx= 3
chemical   H 2 O   c_vec  [0. 2. 1.]
chemical is  S O 2 
c=  ['S', 'O', '2']  c_size= 3
atom is:  S
chemical  S {'S': 1} c_idx= 1
atom is:  O
chemical  O {'S': 1, 'O': 2} c_idx= 3
chemical   S O 2   c_vec  [1. 0. 2.]
chemical is  O 2 
c=  ['O', '2']  c_size= 2
atom is:  O
chemical  O {'O': 2} c_idx= 2
chemical   O 2   c_vec  [0. 0. 2.]
chemical is  H 2 S O 4 
c=  ['H', '2', 'S', 'O', '4']  c_size= 5
atom is:  H
chemical  H {'H': 2} c_idx= 2
atom is:  S
chemical  S {'H': 2, 'S': 1} c_idx= 3
atom is:  O
chemical  O {'H': 2, 'S': 1, 'O': 4} c_idx= 5
chemical   H 2 S O 4   c_vec  [1. 2. 4.]
The solution (as a vector) for the above equation is:
Matrix([
[1.0],
[1.0],
[0.5],
[  1]])
The balanced chemical equation is:
1.00000000000000  H 2 O  + 1.00000000000000  S O 2  + 0.500000000000000  O 2  = 1  H 2 S O 4 


## Task-4: If the above is working, can you wrap the whole program in a single function, which will take the input and give the proper output?

Given the string 'eqn', the output of the function that you will write, called BalanceChemicalEquation(eqn), will be the balanced chemical equation or a proper error

In [16]:
def BalanceChemicalEquation(eqn):
    atom_list = list(set(filter(None, re.split("[ 0-9+=]+", eqn))))
    reactants, products = re.split("=", eqn)
    reacts = reactants.split(sep="+")
    prods = products.split(sep="+")
    try:
        return balance_react_and_prod(reacts, prods, atom_list)[1]
    except ValueError:
        return "ERROR: Could not balance chemical equation!"


In [17]:
print("Sample test case:")
ret = BalanceChemicalEquation("H 2 + O 2 = H 2 O")
print("The balanced equation is:")
print(ret)

Sample test case:
chemical is H 2 
c=  ['H', '2']  c_size= 2
atom is:  H
chemical  H {'H': 2} c_idx= 2
chemical  H 2   c_vec  [2. 0.]
chemical is  O 2 
c=  ['O', '2']  c_size= 2
atom is:  O
chemical  O {'O': 2} c_idx= 2
chemical   O 2   c_vec  [0. 2.]
chemical is  H 2 O
c=  ['H', '2', 'O']  c_size= 3
atom is:  H
chemical  H {'H': 2} c_idx= 2
atom is:  O
chemical  O {'H': 2, 'O': 1} c_idx= 3
chemical   H 2 O  c_vec  [2. 1.]
The balanced equation is:
1.00000000000000 H 2  + 0.500000000000000  O 2  = 1  H 2 O
